## DATASCIENCENIGERIA:  PREDICTION FOR THE BOOTCAMP


In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
X = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')

In [3]:
X.head()

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
3,YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
4,YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


DATA CLEANING

In [4]:
#obtaining the number of rows
print(X.shape)
print(X_test.shape)

(38312, 19)
(16496, 18)


In [5]:
#Getting the columns with missing values
X.isnull().sum()

EmployeeNo                                0
Division                                  0
Qualification                          1679
Gender                                    0
Channel_of_Recruitment                    0
Trainings_Attended                        0
Year_of_birth                             0
Last_performance_score                    0
Year_of_recruitment                       0
Targets_met                               0
Previous_Award                            0
Training_score_average                    0
State_Of_Origin                           0
Foreign_schooled                          0
Marital_Status                            0
Past_Disciplinary_Action                  0
Previous_IntraDepartmental_Movement       0
No_of_previous_employers                  0
Promoted_or_Not                           0
dtype: int64

In [6]:
X_test.isnull().sum()

EmployeeNo                               0
Division                                 0
Qualification                          730
Gender                                   0
Channel_of_Recruitment                   0
Trainings_Attended                       0
Year_of_birth                            0
Last_performance_score                   0
Year_of_recruitment                      0
Targets_met                              0
Previous_Award                           0
Training_score_average                   0
State_Of_Origin                          0
Foreign_schooled                         0
Marital_Status                           0
Past_Disciplinary_Action                 0
Previous_IntraDepartmental_Movement      0
No_of_previous_employers                 0
dtype: int64

In [7]:
X['Qualification'].value_counts()

First Degree or HND         25578
MSc, MBA and PhD            10469
Non-University Education      586
Name: Qualification, dtype: int64

In [8]:
X = X.replace(to_replace=['Non-University Education', 'First Degree or HND', 'MSc, MBA and PhD'], value=[0, 1, 2])
X_test = X_test.replace(to_replace=['Non-University Education', 'First Degree or HND', 'MSc, MBA and PhD'], value=[0, 1, 2])

In [9]:
X.groupby('Promoted_or_Not').mean()

,Qualification,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average
Promoted_or_Not,,,,,,,,
0,1.266565,2.258191,1986.167517,7.562730,2013.123777,0.320379,0.013943,54.641584
1,1.303894,2.204875,1986.661833,9.173095,2013.311941,0.705955,0.122802,63.210429


In [10]:
y = X['Promoted_or_Not']
X.drop(['Promoted_or_Not'], axis=1, inplace=True)

In [11]:
categorical_cols = ['Division']

In [12]:
numerical_cols = ['Training_score_average', 'Last_performance_score', 'Qualification', 
                  'Targets_met', 'Previous_Award', 'Trainings_Attended']

In [14]:
numerical_transformer = SimpleImputer(strategy='median')

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [15]:
model = XGBClassifier(n_estimators=1000, learning_rate=0.07)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

#fitting the model and using it to predict X_test
clf.fit(X_train, y_train)
pred_test = clf.predict(X_valid)

print("Model Accuracy: ", accuracy_score(y_valid, pred_test))

Model Accuracy:  0.9466266475270781


In [21]:
clf.fit(X, y)
predictions = clf.predict(X_test)

In [19]:
output = pd.DataFrame()
output['EmployeeNo'] = X_test['EmployeeNo']
output['Promoted_or_Not'] = predictions

output.to_csv('submission.csv', index=False)
output.head()

,EmployeeNo,Promoted_or_Not
0,YAK/S/00005,0
1,YAK/S/00011,0
2,YAK/S/00015,0
3,YAK/S/00016,0
4,YAK/S/00017,0
